### Visual Display Framework

In [1]:
### Import Libraries ###
# set qt(5) display framework, matplotlib (imshow) for frame display/update
%matplotlib qt
import matplotlib.pyplot as plt
#from jupyterthemes import jtplot; jtplot.style()
import matplotlib.colors as colors
import matplotlib.cm as cmap

### Initialise Interactive Display Figure ###
## Inputs ##
    # ax_rng - axes display range [scalar] relative to zero
## Outputs ##
    # fig - matplotlib.pyplot figure reference
    # axs - figure axes reference
def init_display(_ax_rng = False):
    # ensure set interactive plotting is on, initialise display figure and axes
    plt.ion()
    fig = plt.figure(figsize = (5, 5))
    axs = fig.add_subplot(111)
    # clean format display, no ticks / labels
    #axs.set_xticklabels(''); axs.set_yticklabels('')
    #axs.set_xticks([]); axs.set_yticks([])
    # if input, set axes range (relative zero)
    if _ax_rng:
        axs.set_xlim(-_ax_rng, _ax_rng); axs.set_ylim(-_ax_rng, _ax_rng)
    # return figure and axes
    return fig, axs


### Point Physics Simulation Framework

In [2]:
## import libraries ##
# numerical image array handling / manipulation
import numpy as np

## Generate Node State Information ##
## Inputs
    # n - number of nodes [integer]
    # d - number of spatial dimension [integer]
## Outputs
    # state - node state information [dict of np.array]
def gen_state(_n = 10, _d = 2, _type = 'random'):
    state = {}
    if _type == 'grid':
        #rng = (n**0.5)/2
        state['pos'] = np.array([ [i, j] for i in range(-int(_n/2), int(_n/2)) for j in range(-int(_n/2), int(_n/2)) ], dtype = np.float32) * 10 + (np.random.rand(_n**2, _d) - 0.5) * 2
        state['vel'] = np.zeros( (_n**2, _d) ) + (np.random.rand(_n**2, _d) - 0.5) * .2
        state['acc'] = np.zeros( (_n**2, _d) ) + (np.random.rand(_n**2, _d) - 0.5) * .02
    else:
        # generate _n node random position, velocity, acceleration with _d dimensions
        state['pos'] = (np.random.rand(_n, _d) - 0.5) * 100.
        state['vel'] = (np.random.rand(_n, _d) - 0.5) * 1.
        state['acc'] = (np.random.rand(_n, _d) - 0.5) * .1
    # return generated node state, dict of arrays
    return state

## Generate Node Properties ##
## Inputs
    # n - number of nodes [integer]
## Outputs
    # prop - node properties [dict of np.array]
def gen_prop(_n = 10, _type = 'random'):
    prop = {}
    # generate _n random mass
    if _type == 'grid':
        prop['mass'] = np.random.rand(_n**2) * 20 + 100
    else:
        prop['mass'] = np.random.rand(_n) * 20 + 100
    # return generated node properties, dict of arrays
    return prop


In [3]:
## Calculate Node-Node Euclidean Distance ##
## Inputs
    # pos - node position vectors [array]
## Outputs
    # prop - node-node distances [array]
def get_dist(_pos):
    dist = []
    # iterate each node
    for i in range(len(_pos)):
        _dist = []
        # iterate each node, skip self-relation
        for j in range(len(_pos)):
            if j != i:
                # calculate node-node euclidean distance, store in list
                _dist.append( sum( [ (pos[j][k] - pos[i][k])**2 for k in range(len(pos[i])) ] )**.5 )
        # store each node distance list in list
        dist.append(_dist)
    # return stacked node-node distance lists as array
    return np.stack(dist)

## Calculate Node-Node Distance Vector ##
## Inputs
    # pos - node position vectors [array]
## Outputs
    # prop - node-node distance vectors [array]
def get_dist_vect(_pos):
    dist = []
    # iterate each node
    for i in range(len(_pos)):
        _dist = []
        # iterate each node, skip self-relation
        for j in range(len(_pos)):
            if j != i:
                # store node-node distance vector in list
                _dist.append( pos[j] - pos[i] )
        # store each node distance vector list in list
        dist.append(_dist)
    # return stacked node-node distance vector lists as array
    return np.stack(dist)


In [4]:
### Physics and Parameter Calculation Functions ###

## Calculate Node-Node Gravity Force Vector ##
## Inputs
    # pos - node position vectors [array]
## Outputs
    # prop - node-node distance vectors [array]
def get_grav(_state, _prop, _index, _G = 5.):
    # get node position, mass array by index
    pos = _state['pos'][_index]
    mass = _prop['mass'][_index]
    grav = []
    # iterate each node
    for i in range(len(_index)):
        _grav = []
        # iterate each node, skip self-relation
        for j in range(len(_index)):
            if j != i:
                # calculate inter-node distance and distance vector
                dist_vect = pos[j] - pos[i]
                dist = np.sum( (dist_vect)**2 )**.5
                vect = dist_vect / np.sum( np.abs(dist_vect) )
                # calculate gravity force vector, store in list
                grav_force = _G * (mass[i] * mass[j]) / dist**2
                _grav.append( vect * grav_force )
        # store each gravity force vector list in list
        grav.append(_grav)
    # stack gravity force vector lists as array
    grav = np.stack(grav)
    # return node gross and net gravity force as array
    grav_gross = np.sum(np.abs(grav), axis = 1)
    grav_net = np.sum(grav, axis = 1)
    return grav_net, grav_gross

def get_drag(_state, _index, _D = .1):
    # get node velocity vector
    vel = _state['vel'][_index]
    vect = vel / np.sum( np.abs(vel) )
    # calculate drag force
    drag_force = _D * vel**2
    drag_net = -vect * drag_force
    # return node net drag force as array
    return drag_net

# update node acceleration vector from net force vectors
def get_acc(_state, _prop, _index):
    # get forces
    grav_net, grav_gross = get_grav(_state, _prop, _index, 2.)
    drag_net = get_drag(_state, _index, 5.)
    # calculate force totals
    force_net = grav_net + drag_net - np.array([0., 10.**3])
    force_gross = grav_gross + (-drag_net)
    # calculate acceleration
    mass = _prop['mass'][_index]
    acc = force_net / np.repeat( np.reshape(mass, (-1, 1)), force_net.shape[-1], axis = 1 )
    # return updated acceleration and forces
    return force_net, force_gross, acc

# update node velocity and position vectors from acceleration vector
def get_vel_pos(_state, _prop, _index, _delta):
    # get total force and acceleration, velocity, position
    force_net, force_gross, acc = get_acc(_state, _prop, _index)
    vel = _state['vel'][_index,:]
    pos = _state['pos'][_index,:]
    # calculate updated velocity, position   
    pos += vel * _delta + .5 * acc * _delta**2
    vel += acc * _delta
    # return updated node state
    return force_net, force_gross, acc, vel, pos


In [12]:
## testing

_n = 4; _d = 2
_state = gen_state(_n, _d, _type = 'grid')
_prop = gen_prop(_n, _type = 'grid')

#_index = list(range(5))
_index = list(range(_n**2))
_delta = 0.05
_steps = 200

# initialise figure
fig, axs = init_display(400)

pos = _state['pos'][_index,:]
acc = _state['acc'][_index,:]
mass = _prop['mass'][_index]

force_net, force_gross, acc, vel, pos = get_vel_pos(_state, _prop, _index, _delta)
_state['pos'][_index,:] = pos
_state['vel'][_index,:] = vel
_state['acc'][_index,:] = acc

def get_col(_var):
    #c = np.log( np.sum(np.abs(_var), axis = -1) )
    c = np.sum(np.abs(_var), axis = -1)
    c -= np.min(c)
    c /= 3000
    return cmap.magma(c)

col = get_col(force_gross)

sca = axs.scatter(pos[:,0], pos[:,1], s = mass, c = col)

if True:
    for i in range(_steps):

        force_net, force_gross, acc, vel, pos = get_vel_pos(_state, _prop, _index, _delta)
        _state['pos'][_index,:] = pos
        _state['vel'][_index,:] = vel
        _state['acc'][_index,:] = acc

        if i%1 == 0:
            sca.set_offsets( pos )
            sca.set_color( get_col(force_gross) )
            #sca.set_sizes([i for i in z])

        plt.pause(60**-1)



In [8]:
_prop

{'mass': array([118.69075892, 115.04777422, 101.49807081, 108.17077904,
        108.13018647, 102.28731351, 102.17774767, 110.91321728,
        119.84308209, 106.44058956, 110.08284212, 114.50982683,
        107.60336668, 112.89395482, 111.23800223, 108.41827046,
        106.52470279, 111.54174812, 112.78850958, 114.09889737,
        100.09798711, 104.15186728, 113.91472469, 119.06523252,
        108.77710007, 100.39590035, 102.66821484, 115.06720359,
        116.80920885, 100.44148246, 108.80753243, 101.74333234,
        110.98299931, 103.07481005, 114.10489776, 114.88593264])}